<a href="https://colab.research.google.com/github/elenadanchenkomn/data_analysis/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%B5%D0%B9%D1%81_%D0%9B%D0%BE%D0%B3%D0%B8%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [1]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2026-01-19 07:33:50--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2026-01-19 07:33:51--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log’

auto_purchase.log   100%[===================>] 448.65K  --.-KB/s    in 0.02s   

2026-01-19 07:33:51 (18.1

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [2]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[400:550]:
    print(line)

INFO  | 2022-11-26 00:01:03,024  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-26, количество людей с автопродлением подписки: 0

INFO  | 2022-11-26 00:01:03,027  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-27 00:01:02,455  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-27 00:01:02,480  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-27, количество людей с автопродлением подписки: 0

INFO  | 2022-11-27 00:01:02,483  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-28 00:01:02,406  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-28 00:01:02,424  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-28, количество людей с автопродлением подписки: 0

INFO  | 2022-11-28 00:01:02,427  | f

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [7]:
def count_success_and_failure(file_path):
    attempts = 0
    failures = 0

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if 'Обновляем подписку пользователю' in line:
                attempts += 1

            if line.startswith('ERROR') and 'ошибка при списании' in line:
                failures += 1

    successes = attempts - failures
    return successes, failures


count_success_and_failure('auto_purchase.log')



(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [8]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [9]:

from datetime import datetime
from statistics import median

def auto_renewal_sub(log_file_path):
    # 1) Собираем по каждому дню максимум "кол-ва людей с автопродлением"
    daily_max = {}  # date -> max_count

    with open(log_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if 'количество людей с автопродлением подписки:' not in line:
                continue

            try:
                # Пример фрагмента:
                # "... Cегодня 2022-12-21, количество людей ...: 0"
                after_today = line.split('Cегодня', 1)[1].strip()
                date_str = after_today.split(',', 1)[0].strip()  # "2022-12-21"
                count = int(line.rsplit(':', 1)[1].strip())
                day = datetime.strptime(date_str, '%Y-%m-%d').date()
            except Exception:
                # если вдруг встретится нестандартная строка — пропускаем
                continue

            if day not in daily_max or count > daily_max[day]:
                daily_max[day] = count

    # 2) Упорядочиваем значения по дате
    days_sorted = sorted(daily_max.keys())
    values = [daily_max[d] for d in days_sorted]

    # 3) Считаем сглаживания "только прошлое + текущее"
    moving_avg = []
    median_smooth = []

    running_sum = 0
    prefix = []

    for i, v in enumerate(values, start=1):
        running_sum += v
        prefix.append(v)

        moving_avg.append(round(running_sum / i, 2))
        median_smooth.append(median(prefix))

    # 4) Записываем результат
    with open('auto_renewal_sub.txt', 'w', encoding='utf-8') as out:
        out.write(f"Среднее: {moving_avg}\n")
        out.write(f"Медиана: {median_smooth}\n")

    return moving_avg, median_smooth


auto_renewal_sub('auto_purchase.log')


([1.0,
  0.5,
  0.33,
  0.25,
  0.2,
  0.17,
  0.14,
  0.12,
  0.11,
  0.1,
  0.09,
  0.08,
  0.08,
  0.07,
  0.07,
  0.06,
  0.06,
  0.06,
  0.05,
  0.05,
  0.05,
  0.05,
  0.04,
  0.04,
  0.04,
  0.04,
  0.04,
  0.04,
  0.03,
  0.03,
  0.06,
  0.09,
  0.09,
  0.09,
  0.09,
  0.08,
  0.08,
  0.08,
  0.08,
  0.07,
  0.07,
  0.07,
  0.12,
  0.14,
  0.13,
  0.13,
  0.13,
  0.15,
  0.16,
  0.16,
  0.16,
  0.15,
  0.15,
  0.15,
  0.15,
  0.14,
  0.16,
  0.17,
  0.17,
  0.17,
  0.2,
  0.21,
  0.21,
  0.2,
  0.22,
  0.21,
  0.21,
  0.21,
  0.2,
  0.2,
  0.2,
  0.19,
  0.21,
  0.2,
  0.2,
  0.2,
  0.19,
  0.21,
  0.22,
  0.21,
  0.22,
  0.22,
  0.22,
  0.21,
  0.21,
  0.21,
  0.22,
  0.22,
  0.21,
  0.22,
  0.24,
  0.24,
  0.25,
  0.24,
  0.24,
  0.25,
  0.25,
  0.24,
  0.24,
  0.25,
  0.25,
  0.25,
  0.24,
  0.24,
  0.24,
  0.24,
  0.23,
  0.23,
  0.23,
  0.23,
  0.23,
  0.23,
  0.23,
  0.23,
  0.23,
  0.23,
  0.24,
  0.24,
  0.24,
  0.24,
  0.24,
  0.24,
  0.24,
  0.24,
  0.24,
  0.24,
  0.

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [10]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

--2026-01-19 07:41:43--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2026-01-19 07:41:43--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: ‘cor_auto_renewal.txt.1’

cor_auto_renewal.tx 100%[===================>]   2.37K  --.-KB/s    in 0s      

2026-01-19 07

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [11]:
from datetime import datetime

def sub_renewal_by_day(file_path):
    weekdays = {
        0: "Понедельник",
        1: "Вторник",
        2: "Среда",
        3: "Четверг",
        4: "Пятница",
        5: "Суббота",
        6: "Воскресенье",
    }

    attempts_by_day = {name: 0 for name in weekdays.values()}
    failures_by_day = {name: 0 for name in weekdays.values()}

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # время операции лежит во 2-м "столбце" между |
            parts = line.split("|")
            if len(parts) < 2:
                continue

            dt_raw = parts[1].strip()  # "2023-01-17 00:12:36,045"
            dt_str = dt_raw.split(",", 1)[0]  # "2023-01-17 00:12:36"

            try:
                dt = datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S")
            except ValueError:
                continue

            day_name = weekdays[dt.weekday()]

            # попытка продления
            if line.startswith("INFO") and "Обновляем подписку пользователю" in line:
                attempts_by_day[day_name] += 1

            # ошибка списания
            if line.startswith("ERROR") and "ошибка при списании" in line:
                failures_by_day[day_name] += 1

    # успешные = попытки - ошибки
    success_by_day = {day: attempts_by_day[day] - failures_by_day[day] for day in attempts_by_day}

    with open("weekdays.txt", "w", encoding="utf-8") as out:
        out.write("Количество обновлений подписки по дням недели:\n\n")
        for i in range(7):
            day = weekdays[i]
            out.write(f"{day}: {success_by_day.get(day, 0)}\n")

    return success_by_day


sub_renewal_by_day("auto_purchase.log")


{'Понедельник': 136,
 'Вторник': 144,
 'Среда': 162,
 'Четверг': 169,
 'Пятница': 145,
 'Суббота': 135,
 'Воскресенье': 143}

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [12]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

--2026-01-19 07:42:54--  https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt [following]
--2026-01-19 07:42:54--  https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238 [text/plain]
Saving to: ‘cor_weekdays.txt’

cor_weekdays.txt    100%[===================>]     238  --.-KB/s    in 0s      

2026-01-19 07:42:55 (5.37 MB/s) - ‘cor_

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
